In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.utils.data as dutils

## CNN

#### Layer 1
28x28x1 → CONV(5, ch=10) → 24x24x10 → MAXPOOL(2) → 12x12x10 → RELU → 12x12x10
####   
#### Layer 2
12x12x10 → CONV(5, ch=20) → 8x8x20 → DROPOUT → 8x8x20 → MAXPOOL(2) → 4x4x20 → RELU → 4x4x20
####  
#### Layer 3
4x4x20 → FLATTEN → 320 → LINEAR(320, 50) → 50 → RELU → 50 → DROPOUT → 50
####  
#### Layer 4
50 → LINEAR(50, 10) → 10 → LOG-SOFTMAX → 10

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(F.dropout(self.conv2(x), training=self.training), 2))
        x = F.dropout(F.relu(self.fc1(x.view(-1, 320))), training=self.training)
        x = F.log_softmax(self.fc2(x), dim=1)
        return x

In [3]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
xforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])
trainset = datasets.MNIST('/data/pytorch/mnist/', download=True, train=True, transform=xforms)
trainloader = dutils.DataLoader(trainset, batch_size=64, shuffle=True)
train_image, train_label = trainset[0]
print(train_image.size())
print(train_label)

testset = datasets.MNIST('/data/pytorch/mnist/', download=True, train=False, transform=xforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

torch.Size([1, 28, 28])
tensor(5)


In [ ]:
model = Net()
model = model.to(DEVICE)
loss_fn = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train
num_epochs = 10
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    for images, labels in trainloader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.enable_grad():
            log_probs = model(images)
            loss = loss_fn(log_probs, labels)
            loss.backward()
            optimizer.step()

In [ ]:
# Test
test_acc = 0
for images, labels in testloader:
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)
    with torch.no_grad():
        log_probs = model(images)
        preds = torch.argmax(log_probs, dim=1)
        batch_acc = torch.sum(preds == labels.data)
        test_acc += batch_acc
avg_test_acc = test_acc.double() / len(testset)
print(f'Test Accuracy: {avg_test_acc:.3f}')